In [ ]:
import requests, random
import gzip, json
from io import BytesIO

#2 step randomization (option1 discussed inclass)
#   2.1 - extract a random list of urls from dolma 
#   2.2 - filter out urls containing unsafe files (refinedweb ones)
#   2.3 - true random sampling of 15 urls to extract tokens from (next cell)

#full list of Dolma dataset files are here...
dolma_url_list = "https://huggingface.co/datasets/allenai/dolma/raw/main/urls/v1_7.txt"

#download url list via python request
resp = requests.get(dolma_url_list)
all_urls = resp.text.splitlines()  #break list into separate urls like [ 'https://url1', 'https://url2', ..., 'https://urln']

#filter out unsafe or unwanted sources
safe_urls = [u for u in all_urls if "falcon-refinedweb" not in u]

#randomly sample 15
sampled_urls = random.sample(safe_urls, 15)
# print("Selected urls:\n")
# for u in sampled_urls:
#     print(u)

#stream 300k tokens from 15 randomly selected urls 
token_target = 300000
token_count = 0
tokens_collected = [] #stores the actual text samples extracted in an array

#loop runs until the token_target is reached
# - the above usually fulfilled from 1st file in loop
# - since a different random sample of files is selected each time 
#       -- high possibility that a different file is being tokenized each time - 2nd layer of randomization
for i, data_url in enumerate(sampled_urls):
    print(f"\nReading file at ->>> {data_url}")
    resp = requests.get(data_url, stream=True)
    resp.raise_for_status()

    #open compressed stream
    with gzip.open(BytesIO(resp.content), "rt", encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
                text = obj.get("text", "")
                tokens = len(text.split())
                token_count += tokens
                tokens_collected.append(text)

                if token_count >= token_target:
                    print(f"\nGood job! Reached target. Collected {token_count} tokens")
                    break
            except json.JSONDecodeError:
                continue
    if token_count >= token_target:
        break

print(f"\nCollected {len(tokens_collected)} text samples from (~{token_count} tokens).")


Testing

In [ ]:
#view collected text
print("\nExtracted text samples\n")
for i, sample in enumerate(tokens_collected, 1):
    print(f"sample #{i}:\n{sample}\n\n")